# **Distribución de la población por género y rango de edad**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_" y contienen "poblacion_vallarta"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    if nombre.startswith("df_") and 'poblacion_vallarta' in nombre
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

Lista de DataFrames filtrados:


['df_poblacion_vallarta']

In [25]:
df_poblacion_vallarta.columns

Index(['id', 'Age Range', 'Hombre', 'Mujer', 'Total'], dtype='object')

In [26]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'Hombre':'hombres','Mujer':'mujeres', 'Total':'total','Age Range':'rango'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


Columnas renombradas en los DataFrames filtrados.


In [27]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id', 'rango', 'total', 'hombres', 'mujeres']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)

In [28]:
dfs = []
for df in dataframes_list:
    dfs.append(df)
dfs = pd.concat(dfs, ignore_index=True)
dfs

,id,rango,total,hombres,mujeres
0,1,0 a 4 años,23198,11680,11518
1,2,10 a 14 años,24823,12592,12231
2,3,15 a 19 años,24230,12248,11982
3,4,20 a 24 años,26579,13436,13143
4,5,25 a 29 años,26132,13145,12987
5,6,30 a 34 años,24517,12274,12243
6,7,35 a 39 años,22319,11317,11002
7,8,40 a 44 años,19862,9928,9934
8,9,45 a 49 años,18621,9258,9363
9,10,5 a 9 años,24049,12298,11751


In [29]:
dfs['limite_inferior'] = df['rango'].apply(lambda x: int(x.split()[0]))
# Ordenar el DataFrame por el límite inferior
dfs = dfs.sort_values(by='limite_inferior').reset_index(drop=True)
dfs['ids'] = range(1, len(dfs) + 1)
dfs

,id,rango,total,hombres,mujeres,limite_inferior,ids
0,1,0 a 4 años,23198,11680,11518,0,1
1,10,5 a 9 años,24049,12298,11751,5,2
2,2,10 a 14 años,24823,12592,12231,10,3
3,3,15 a 19 años,24230,12248,11982,15,4
4,4,20 a 24 años,26579,13436,13143,20,5
5,5,25 a 29 años,26132,13145,12987,25,6
6,6,30 a 34 años,24517,12274,12243,30,7
7,7,35 a 39 años,22319,11317,11002,35,8
8,8,40 a 44 años,19862,9928,9934,40,9
9,9,45 a 49 años,18621,9258,9363,45,10


In [30]:
rangos_edad = {
    "0-19": (1, 4),
    "20-34": (5, 7),
    "35-49": (8,10),
    "50-64": (11,13),
    "65+":(14, float('inf'))
}
# Función para asignar la categoría según el rango de edad
def asignar_categoria(id_value):
    # Usar directamente el valor de ids (límite inferior)
    for categoria, (limite_inferior, limite_superior) in rangos_edad.items():
        if limite_inferior <= id_value <= limite_superior:
            return categoria
    return None  # En caso de que el id no caiga en ningún rango definido

# Asignar la categoría a cada registro
dfs['rango_amplio'] = dfs['ids'].apply(asignar_categoria)
dfs

,id,rango,total,hombres,mujeres,limite_inferior,ids,rango_amplio
0,1,0 a 4 años,23198,11680,11518,0,1,0-19
1,10,5 a 9 años,24049,12298,11751,5,2,0-19
2,2,10 a 14 años,24823,12592,12231,10,3,0-19
3,3,15 a 19 años,24230,12248,11982,15,4,0-19
4,4,20 a 24 años,26579,13436,13143,20,5,20-34
5,5,25 a 29 años,26132,13145,12987,25,6,20-34
6,6,30 a 34 años,24517,12274,12243,30,7,20-34
7,7,35 a 39 años,22319,11317,11002,35,8,35-49
8,8,40 a 44 años,19862,9928,9934,40,9,35-49
9,9,45 a 49 años,18621,9258,9363,45,10,35-49


In [31]:
# Agrupar y sumar en una línea
datos_agrupados = dfs.groupby('rango_amplio').sum(numeric_only=False).reset_index()
datos_agrupados

,rango_amplio,id,rango,total,hombres,mujeres,limite_inferior,ids
0,0-19,16,0 a 4 años5 a 9 años10 a 14 años15 a 19 años,96300,48818,47482,30,10
1,20-34,15,20 a 24 años25 a 29 años30 a 34 años,77228,38855,38373,75,18
2,35-49,24,35 a 39 años40 a 44 años45 a 49 años,60802,30503,30299,120,27
3,50-64,36,50 a 54 años55 a 59 años60 a 64 años,39084,19114,19970,165,36
4,65+,80,65 a 69 años70 a 74 años75 a 79 años80 a 84 añ...,18347,8859,9488,375,80


### 2022

In [ ]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

data = {
    'modalidad': ['Nueva','Nueva','Nueva','Nueva', 'Usada', 'Usada', 'Usada', 'Usada'],
    'trimestre': ['1T 2022','2T 2022','3T 2022','4T 2022','1T 2022','2T 2022','3T 2022','4T 2022'],
    'promedio': ['1,533,116.58', '1,584,256.88', '1,564,172.85', '1,915,828.41', '1,805,506.13', '1,911,416.26', '2,253,129.77', '2,372,802.68'],
}
# Convertir a DataFrame
promedio_monto_por_modalidad_mes = pd.DataFrame(data)

In [ ]:
# Datos
trimestres = promedio_monto_por_modalidad_mes['trimestre']
modalidades = promedio_monto_por_modalidad_mes['modalidad']
promedios = promedio_monto_por_modalidad_mes['promedio']

# Limpiar las cadenas de promedio y convertirlas a números
promedios_limpios = [float(promedio.replace(',', '')) for promedio in promedios]

colors = ['#9500ff', '#ff0059']
fig = go.Figure()

# Agregar líneas para cada modalidad
for i, modalidad in enumerate(modalidades.unique()):
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == modalidad]
    fig.add_trace(go.Scatter(
        x=df_modalidad['trimestre'],
        y=df_modalidad['promedio'].apply(lambda x: float(x.replace(',', ''))),  # Usar los promedios limpios
        mode='markers+lines',
        name=modalidad,
        line=dict(color=colors[i % len(colors)])  # Asignar colores a las líneas
    ))

fig.update_layout(
    #title='Promedio de valor de créditos para viviendas nuevas y usadas (2023)',
    #xaxis=dict(title='Trimestre'),
    yaxis=dict(title='Promedio del monto', gridcolor='#dddcda'),
    margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Agregar etiquetas de texto a los puntos
for trace in fig.data:
    df_modalidad = promedio_monto_por_modalidad_mes[promedio_monto_por_modalidad_mes['modalidad'] == trace.name]
    for i, point in enumerate(trace.y):
        fig.add_annotation(
            x=trace.x[i],
            y=point,
            text=f'{df_modalidad.iloc[i]["promedio"]}',
            showarrow=False,
            font=dict(color='black', size=10),
            yshift=10
        )

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets\graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
        
    # Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }

    # Guardar la gráfica como archivo HTML
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)

guardar_grafico_como_html(fig, 'g_scatt_creditos_nuevausada_2022', carpeta='assets\graficas')
fig.show()

<>:45: SyntaxWarning:

invalid escape sequence '\g'

<>:64: SyntaxWarning:

invalid escape sequence '\g'

<>:45: SyntaxWarning:

invalid escape sequence '\g'

<>:64: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_18196\3069977503.py:45: SyntaxWarning:

invalid escape sequence '\g'

C:\Users\yoe11\AppData\Local\Temp\ipykernel_18196\3069977503.py:64: SyntaxWarning:

invalid escape sequence '\g'



In [32]:
import plotly.graph_objects as go

# Crear una gráfica de pirámide de población
def crear_grafico_piramide(dfs):
    fig = go.Figure()

    # Sumar los valores de hombres y mujeres por rango amplio
    datos_agrupados = dfs.groupby('rango_amplio').sum(numeric_only=False).reset_index()
    

    # Calcular el total de hombres y mujeres
    total_hombres = datos_agrupados['hombres'].sum()
    total_mujeres = datos_agrupados['mujeres'].sum()
    total_poblacion = total_hombres + total_mujeres

    # Crear texto para el hover que incluya total y porcentaje
    text_hombres = [
        f'Total: {row["hombres"]}<br>Porcentaje: {row["hombres"] / total_poblacion * 100:.1f}%'
        for index, row in datos_agrupados.iterrows()
    ]

    text_mujeres = [
        f'Total: {row["mujeres"]}<br>Porcentaje: {row["mujeres"] / total_poblacion * 100:.1f}%'
        for index, row in datos_agrupados.iterrows()
    ]

    # Crear barras para hombres a la izquierda
    fig.add_trace(go.Bar(
        y=datos_agrupados['rango_amplio'],
        x=-datos_agrupados['hombres'],  # Mantener la barra en negativo para la forma de pirámide
        orientation='h',
        marker=dict(color='#9500ff'),
        text=text_hombres,
        textposition='none',  # Ocultar el texto en las barras
        hoverinfo='text',  # Usar solo el texto personalizado al hacer hover
        name='Hombres'
    ))

    # Crear barras para mujeres a la derecha
    fig.add_trace(go.Bar(
        y=datos_agrupados['rango_amplio'],
        x=datos_agrupados['mujeres'],
        orientation='h',
        marker=dict(color='#ff0059'),
        text=text_mujeres,
        textposition='none',  # Ocultar el texto en las barras
        hoverinfo='text',  # Usar solo el texto personalizado al hacer hover
        name='Mujeres'
    ))

    # Configurar el gráfico
    fig.update_layout(
        barmode='overlay',
        bargap=0.35,
        xaxis=dict(
            title='Población',
            gridcolor='#dddcda',  # Color de las líneas que dividen los rangos del eje Y
            tickvals=[-50000, -40000, -30000, -20000, -10000, 0, 10000, 20000, 30000, 40000, 50000],  # Ajusta según tus datos
            ticktext=[50000, 40000, 30000, 20000, 10000, 0, 10000, 20000, 30000, 40000, 50000]  # Etiquetas positivas
        ),
        plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
        margin=dict(l=10, r=10, t=10, b=10),  # Ajusta los márgenes (left, right, top, bottom)
        yaxis=dict(title='Rango de Edad'),
        legend=dict(
            font_size=10,
            orientation='h',
            x=1,  # Posición horizontal (1 es el extremo derecho)
            y=1,  # Posición vertical (1 es la parte superior)
            xanchor='right',  # Ancla la leyenda a la derecha
            yanchor='top'  # Ancla la leyenda en la parte superior
        )
    )

    return fig


def guardar_grafico_como_html(fig, nombre_archivo, carpeta='assets/graficas'):
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    # Configuración personalizada para la gráfica
    config = {
        'displaylogo': False,  # Ocultar el logo de Plotly
        'modeBarButtonsToRemove': [
            'toImage',       # Botón para guardar como imagen
            'select2d',      # Box select
            'lasso2d',       # Lasso select
            'resetScale2d',  # Reset Axes
        ]
    }
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html', config=config)
    #pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Crear la figura de la pirámide de población
fig_piramide = crear_grafico_piramide(dfs)
fig_piramide.show()
# Guardar la figura como archivo HTML
guardar_grafico_como_html(fig_piramide, 'g_piramid_poblacion')